In [1]:
import pandas as pd
from datetime import datetime
from datetime import timezone

In [2]:
calendar_df=pd.read_csv("gtfs/calendar.txt")

In [3]:
calendar_df

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,1,0,0,1,1,1,0,0,20200715,20200718
1,3,0,0,0,0,0,1,0,20200715,20200718
2,y1001,0,0,0,0,0,1,0,20200719,20200815
3,y1002,1,1,1,1,1,0,0,20200719,20200815
4,y1003,1,0,0,0,0,0,1,20200719,20200815


If the most recent calendar should be shown for now, only the following three service_ids will be used in the timetable:

In [4]:
viable_service_lines=["y1002","y1001","y1003"]

In [5]:
trips_df=pd.read_csv("gtfs/trips.txt")

In [6]:
trips_df.head()

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id
0,60-11-b12-1,1,3271.1.60-11-b12-1.16.I,60-11-b12-1.16.I,Blackthorn Road - Saint Pappin's Road,1
1,60-11-b12-1,1,3272.1.60-11-b12-1.15.O,60-11-b12-1.15.O,Saint Pappin's Road - Blackthorn Drive (Blackt...,0
2,60-11-b12-1,1,3273.1.60-11-b12-1.16.I,60-11-b12-1.16.I,Blackthorn Road - Saint Pappin's Road,1
3,60-11-b12-1,1,3274.1.60-11-b12-1.15.O,60-11-b12-1.15.O,Saint Pappin's Road - Blackthorn Drive (Blackt...,0
4,60-11-b12-1,1,3275.1.60-11-b12-1.16.I,60-11-b12-1.16.I,Blackthorn Road - Saint Pappin's Road,1


In [7]:
trips_df=trips_df[trips_df["service_id"].isin(viable_service_lines)]

In [8]:
trips_df=trips_df.drop(["shape_id","trip_headsign"],axis=1)

Direction in Training data also referred to inbound and outbound, but was remapped to 1 and 2. We must find one match in order to determine the mapping between (0,1) and (1,2).

Having compared the data with our historical data, the mapping is {0:1,1:2}.

In [9]:
trips_df["direction_id"]=trips_df["direction_id"]+1

In [10]:
trips_df.head()

,route_id,service_id,trip_id,direction_id
155,60-11-d12-1,y1003,14677.y1003.60-11-d12-1.16.I,2
156,60-11-d12-1,y1003,14678.y1003.60-11-d12-1.15.O,1
157,60-11-d12-1,y1003,14679.y1003.60-11-d12-1.16.I,2
158,60-11-d12-1,y1003,14680.y1003.60-11-d12-1.15.O,1
159,60-11-d12-1,y1003,14681.y1003.60-11-d12-1.16.I,2


In [11]:
routes_df=pd.read_csv("gtfs/routes.txt")

In [12]:
routes_df.head()

,route_id,agency_id,route_short_name,route_long_name,route_type
0,60-1-b12-1,978,1,NaN,3
1,60-102-b12-1,978,102,NaN,3
2,60-104-b12-1,978,104,NaN,3
3,60-11-b12-1,978,11,NaN,3
4,60-111-b12-1,978,111,NaN,3


In [13]:
routes_df=routes_df.drop(["agency_id","route_long_name","route_type"],axis=1)

In [14]:
trips_df.shape

(15724, 4)

In [15]:
trips_df=pd.merge(trips_df, routes_df, on='route_id', how='inner')

In [16]:
trips_df.shape

(15724, 5)

In [17]:
trips_df=trips_df.drop(["route_id"],axis=1)

In [18]:
trips_df=trips_df.rename({"route_short_name":"route_id","direction_id":"direction"},axis="columns")

In [19]:
trips_df["route_id"]=trips_df["route_id"].apply(lambda x:x.upper())

In [20]:
trips_df.head()

,service_id,trip_id,direction,route_id
0,y1003,14677.y1003.60-11-d12-1.16.I,2,11
1,y1003,14678.y1003.60-11-d12-1.15.O,1,11
2,y1003,14679.y1003.60-11-d12-1.16.I,2,11
3,y1003,14680.y1003.60-11-d12-1.15.O,1,11
4,y1003,14681.y1003.60-11-d12-1.16.I,2,11


In [21]:
len(trips_df.route_id.unique())

95

In [22]:
stops_df=pd.read_csv("gtfs/stops.txt")

In [23]:
test_name="Parnell Square West, stop 2"

In [24]:
def name_converter(name):
    name_split=name.split(",")
    try:
        return [name_split[0],int(name_split[-1].split()[-1])]
    except:
        return [None,None]

In [25]:
stops_df["name"],stops_df["new_id"]=zip(*stops_df["stop_name"].apply(lambda x:name_converter(x)))

In [26]:
stops_df.head()

,stop_id,stop_name,stop_lat,stop_lon,name,new_id
0,8220DB000002,"Parnell Square West, stop 2",53.352244,-6.263723,Parnell Square West,2
1,8220DB000003,"Parnell Square West, stop 3",53.352309,-6.263811,Parnell Square West,3
2,8220DB000004,"Parnell Square West, stop 4",53.352575,-6.264175,Parnell Square West,4
3,8220DB000006,"Parnell Square West, stop 6",53.352749,-6.264454,Parnell Square West,6
4,8220DB000007,"Parnell Square West, stop 7",53.352841,-6.264570,Parnell Square West,7


In [27]:
stops_df[stops_df["new_id"].isna()]

,stop_id,stop_name,stop_lat,stop_lon,name,new_id
3172,8240DB007040,Hearse Road R126,53.480551,-6.191769,None,None


could not find a match for this stop, so we drop it.

In [28]:
stops_df=stops_df.dropna()

In [29]:
id_match=stops_df[["stop_id","new_id"]]

In [30]:
id_match.head()

,stop_id,new_id
0,8220DB000002,2
1,8220DB000003,3
2,8220DB000004,4
3,8220DB000006,6
4,8220DB000007,7


In [31]:
times_df=pd.read_csv("gtfs/stop_times.txt")

In [32]:
times_df=times_df.drop(["stop_headsign","pickup_type","drop_off_type","shape_dist_traveled"],axis=1)

In [33]:
times_df=pd.merge(times_df, id_match, on='stop_id', how='inner')

In [34]:
times_df=times_df.drop(["stop_id"],axis=1)

In [35]:
times_df=times_df.rename({"new_id":"stop_id","stop_sequence":"progr_number","arrival_time":"arr","departure_time":"dep"},axis="columns")

In [36]:
def time_converter(time_string):
    hour,minute,second=time_string.split(":")
    hour=int(hour)
    day=1
    if hour>23:
        hour=hour%24
        day+=1
    dt=datetime(year=1970,month=1,day=day,hour=hour,minute=int(minute),second=int(second))
    return int(dt.replace(tzinfo=timezone.utc).timestamp())

In [37]:
times_df["arr"]=times_df["arr"].apply(lambda x:time_converter(x))
times_df["dep"]=times_df["dep"].apply(lambda x:time_converter(x))

In [38]:
diff=times_df.dep-times_df.arr

In [39]:
diff.value_counts()

0    1573335
dtype: int64

all arrivals and departures are identical. thus, one of the features can be dropped.

In [40]:
times_start_df=times_df[["trip_id","dep","progr_number"]][times_df.progr_number==1]

In [41]:
times_start_df=times_start_df.drop(["progr_number"],axis=1)
times_start_df=times_start_df.rename({"dep":"start"},axis=1)

In [42]:
times_df=pd.merge(times_df, times_start_df, on="trip_id", how='inner')

In [43]:
times_df.head()

,trip_id,arr,dep,progr_number,stop_id,start
0,1.1.60-39A-b12-1.205.I,28500,28500,1,7158,28500
1,1.1.60-39A-b12-1.205.I,28543,28543,2,7048,28500
2,1.1.60-39A-b12-1.205.I,28587,28587,3,7699,28500
3,1.1.60-39A-b12-1.205.I,28614,28614,4,7159,28500
4,1.1.60-39A-b12-1.205.I,28633,28633,5,7388,28500


In [44]:
times_df["cum_dur"]=times_df["dep"]-times_df["start"]

In [45]:
times_df=times_df.drop(["arr","start"],axis=1)

In [46]:
times_df.head()

,trip_id,dep,progr_number,stop_id,cum_dur
0,1.1.60-39A-b12-1.205.I,28500,1,7158,0
1,1.1.60-39A-b12-1.205.I,28543,2,7048,43
2,1.1.60-39A-b12-1.205.I,28587,3,7699,87
3,1.1.60-39A-b12-1.205.I,28614,4,7159,114
4,1.1.60-39A-b12-1.205.I,28633,5,7388,133


In [47]:
stops_df=stops_df.drop(["stop_id","stop_name"],axis=1)

In [48]:
stops_df=stops_df.rename({"new_id":"stop_id","stop_lat":"lat","stop_lon":"lon"},axis="columns")

In [49]:
trips_df.head()

,service_id,trip_id,direction,route_id
0,y1003,14677.y1003.60-11-d12-1.16.I,2,11
1,y1003,14678.y1003.60-11-d12-1.15.O,1,11
2,y1003,14679.y1003.60-11-d12-1.16.I,2,11
3,y1003,14680.y1003.60-11-d12-1.15.O,1,11
4,y1003,14681.y1003.60-11-d12-1.16.I,2,11


In [50]:
stops_df.head()

,lat,lon,name,stop_id
0,53.352244,-6.263723,Parnell Square West,2
1,53.352309,-6.263811,Parnell Square West,3
2,53.352575,-6.264175,Parnell Square West,4
3,53.352749,-6.264454,Parnell Square West,6
4,53.352841,-6.264570,Parnell Square West,7


Sanity Check for directions:

In [51]:
times_df.tail()

,trip_id,dep,progr_number,stop_id,cum_dur
1573330,976.y1002.60-44B-d12-1.248.O,64259,25,3507,1259
1573331,976.y1002.60-44B-d12-1.248.O,64354,26,3508,1354
1573332,976.y1002.60-44B-d12-1.248.O,64395,27,3510,1395
1573333,976.y1002.60-44B-d12-1.248.O,64482,28,3512,1482
1573334,976.y1002.60-44B-d12-1.248.O,64529,29,3514,1529


In [52]:
trips_df[trips_df.trip_id=="977.y1002.60-44B-d12-1.249.I"]

,service_id,trip_id,direction,route_id
11103,y1002,977.y1002.60-44B-d12-1.249.I,2,44B


Stop ID 6007, Sequence 17 matches up with direction 2 in historic data.